# Emotion Analysis of Writers and Readers of Japanese Tweets on Vaccinations

Patrick Ramos

NAPI 2022 Intern

---

This notebook performs emotional analysis on a collection of Japanese tweets about vaccines and compares the extracted emotions to vaccination rates in Japan. Emotion mining is performed by fine-tuning a pre-trained BERT on a Japanese emotional analysis dataset and using the fine-tuned model and a dataset of tweets about vaccines to infer the emotions of Japanese SNS users towards vaccines.

Acknowledgements:

* PhD Candidate Kiki Ferawati for the vaccine-related SNS post dataset
* Dr. Aramaki, Dr. Wakamiya, Dr. Kong, Ms. Kiki, Mr. Lean, and Mr. Kiyomoto for advice and guidance regarding the project

## Installations, imports and downloads

### Downloads

The [WRIME dataset](https://github.com/ids-cv/wrime) will be used for fine-tuning BERT on emotion analysis. The dataset will be explained in more detail later.

In [ ]:
!wget https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver1.tsv

Vaccination data is downloaded from [Our World in Data](https://github.com/owid/covid-19-data/tree/master/public/data/vaccinations).

In [ ]:
!wget https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv

### Installations

We will need to install [HuggingFace](https://huggingface.co/) for the BERT model. We might have to restart the runtime after this due to updating matplotlib

In [ ]:
pip install -U transformers['ja'] datasets matplotlib

### Imports

We use several libraries for this project, including PyTorch and HuggingFace for the model training, pandas for cleaning and organizing our data, and plotly for visualizing our results.

In [ ]:
from datasets import load_dataset
import numpy as np
from scipy.stats import pearsonr, ks_2samp
from sklearn import metrics
import torch
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, pipeline
from transformers.pipelines.base import KeyDataset
from tqdm.notebook import tqdm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd

from itertools import chain, product
from pathlib import Path

We can load tensorboard to view model metrics after training.

In [ ]:
%load_ext tensorboard

## Setup for saving

If we would like to save the BERT checkpoints and other datasets, we can mount our Google Drive on this runtime.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Fill out the form below to choose were to save our datasets and model checkpoints.

In [ ]:
save_root = '/content/'#@param {'type':'string'}
save_root = Path(save_root)

## Fine-tuning BERT on emotion analysis

As stated earlier, we will use the [WRIME dataset](https://aclanthology.org/2021.naacl-main.169.pdf) to fine-tune BERT on emotion analysis. WRIME (Dataset of **W**riters’ and **R**eaders’ **I**ntensities of E**m**otion
for their **E**stimation) is a dataset of 40,000 SNS posts annotated with the "subjective" emotional intensities of the posts' writers and the "objective" emotional intensities of the posts' readers.

Each post is annotated with following eight emotions (following Plutchik's framework) for both writers and readers:

* joy
* sadness
* anticipation
* surprise
* anger
* fear
* disgust
* trust

Each post can exhibit one or several of the emotions and the intesities are integers from 0 to 3, inclusive (respectively indicating "none", "weak", "medium", and "strong"). An example showing the annotation of a writer and the annotations of three readers, taken from the paper's Github, is shown below.

"タイヤがパンクしてた。。いたずらの可能性が高いんだって。。"

("The tire of my car was flat. I heard that it might be mischief.")

||Joy	|Sadness	|Anticipation	|Surprise	|Anger	|Fear	|Disgust	|Trust|
|---|---|---|---|---|---|---|---|---|
|Writer	 |0	|3	|0	|1	|3	|0	|0	|0|
|Reader 1|0	|3	|0	|3	|1	|2	|1	|0|
|Reader 2|0	|2	|0	|2	|0	|0	|0	|0|
|Reader 3|0	|2	|0	|2	|0	|1	|1	|0|

As there are three annotators for reader emotion, we average their scores to create only one set of reader emotions per data point. We also split the dataset into train, test, and validation sets using the splits provided by the dataset.

In [ ]:
emotion_names = ('joy', 'sadness', 'anticipation', 'surprise', 'anger', 'fear', 'disgust', 'trust')
wrime_tsv_targets = [f'Writer_{emotion.title()}' for emotion in emotion_names] + \
                    [f'Reader{i}_{emotion.title()}' for i, emotion in product(range(1, 4), emotion_names)]

wrime_df = pd.read_csv('wrime-ver1.tsv', sep='\t')
wrime_df = wrime_df.rename(
    columns=dict([('Train/Dev/Test', 'split'), ('Sentence', 'text')]+\
    [(col, col.replace('Saddness', 'Sadness')) for col in wrime_df.columns if 'Saddness' in col])
)
wrime_df = wrime_df[['split', 'text', *wrime_tsv_targets]]

for emotion in emotion_names:
  wrime_df[f'Reader_{emotion.title()}'] = wrime_df[[f'Reader{i}_{emotion.title()}' for i in range(1, 4)]].mean(axis=1)
annotators = ('writer', 'reader')
wrime_df['label_ids'] = wrime_df[[f'{annotator.title()}_{emotion.title()}' for annotator, emotion in product(annotators, emotion_names)]].values.tolist()
wrime_df = wrime_df[['split', 'text', 'label_ids']]

for df_split, dataset_split in zip(['train', 'dev', 'test'], ['train', 'validation', 'test']):
  wrime_df[wrime_df.split==df_split].drop(columns='split').to_csv(save_root/f'wrime_{dataset_split}.csv', index=False)

We can now load the dataset from the splits.

In [ ]:
wrime = load_dataset('csv', data_files={split:str(save_root/f'wrime_{split}.csv') for split in ['train', 'validation', 'test']})

To use BERT, the inputs must be tokenized. We use a [tokenizer pre-trained on Japanese text by the Tohoku NLP Lab](https://huggingface.co/cl-tohoku/bert-base-japanese-v2) to tokenize WRIME.

In [ ]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')

def preprocess_function(examples):
  examples['label_ids'] = [eval(label_ids) for label_ids in examples['label_ids']]
  examples = tokenizer(examples['text'], truncation=True, max_length=512)
  return examples

tokenized_wrime = wrime.map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=512)

Now we can actually load the BERT model. The BERT model we use is also from the [Tohoku NLP Lab](https://huggingface.co/cl-tohoku/bert-base-japanese-v2) and is pre-trained on Japanese Wikipedia.

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'cl-tohoku/bert-base-japanese-v2',
    num_labels=16,
    problem_type='regression'
)

While we will fine-tune using mean squared error as our loss function, we can also report mean absolute error during training to measure performance.

In [ ]:
def compute_metrics(eval_pred):
  """Computes metrics"""
  logits, labels = eval_pred
  return {
      'mae' : metrics.mean_absolute_error(y_true=labels, y_pred=logits)
  }

Now we can begin training.

In [ ]:
training_args = TrainingArguments(
    output_dir=save_root / 'bert-base-japanese-v2-wrime-fine-tune',
    evaluation_strategy='epoch',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    num_train_epochs=3,
    lr_scheduler_type='linear',
    warmup_ratio=0.01,
    save_strategy='epoch',
    report_to='tensorboard'
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_wrime['train'].shuffle(seed=42),
    eval_dataset=tokenized_wrime['test'].shuffle(seed=42),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

We can also look at our metrics through Tensorboard.

In [ ]:
%tensorboard --logdir "$save_root/bert-base-japanese-v2-wrime-fine-tune/"

## Inferring emotions from vaccine tweets

Thanks to PhD Candidate Kiki Ferawati for providing the vaccine dataset. The dataset is composed of 20,254 tweets containing the keywords `'ワクチン'`, `'モデルナ'`, `'ファイザー'` and/or `オミクロン` scraped from December 1, 2021 to December 31, 2021. For each keyword, 15 random minutes were sampled from each day in December 2021, and all tweets containing the keyword in those minutes were scraped.  We preprocess this dataset by removing `'\r'` characters, which can affect parsing into a dataset compatible with BERT.

We provide the IDs of the Tweets. Scrape the Tweets using your favorite scraper and save them into a file called `vaccine_tweets.csv`. This csv should have the Tweet text in a column called `"text"`.

We can load the preprocessed dataset as follows.

In [ ]:
vaccine_tweets = load_dataset('csv', data_files={'unsupervised':'vaccine_tweets.csv'})['unsupervised']

Again, the emotions we will be extracting from the vaccine-related tweets are:

* joy
* sadness
* anticipation
* surprise
* anger
* fear
* disgust
* trust

In [ ]:
emotion_names = ('joy', 'sadness', 'anticipation', 'surprise', 'anger', 'fear', 'disgust', 'trust')
annotators = ('writer', 'reader')
labels = [f'{annotator}_{emotion}' for annotator, emotion in (product(annotators, emotion_names))]

We load the same tokenizer but use the BERT model we trained earlier.

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    save_root/'bert-base-japanese-v2-wrime-fine-tune/checkpoint-3750',
    num_labels=16,
    id2label={i:label for i, label in enumerate(labels)},
    max_length=512
)
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')

The pipeline shown below will allow the easy inference of emotion scores from input text.

In [ ]:
pipe = pipeline(
    'text-classification',
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True,
    function_to_apply='none',
    device=0 if torch.cuda.is_available() else -1
)

We iterate over the vaccine tweets and infer the emotion scores from each tweet.

In [ ]:
emotion_scores = pd.DataFrame()
for out in tqdm(pipe(KeyDataset(vaccine_tweets, 'text'), batch_size=32)):
  emotion_scores = emotion_scores.append(
      pd.DataFrame([dict([tuple(label_score.values()) for label_score in out])]),
      ignore_index=True
  )

We attach the emotion scores to each the vaccine tweet dataset to more easily view the emotion scores per tweet.

In [ ]:
vaccine_tweets_df = pd.read_csv('vaccine_tweets.csv')
vaccine_tweets_emotions = pd.concat((vaccine_tweets_df, emotion_scores), axis='columns')
vaccine_tweets_emotions.to_csv(save_root/'vaccine_tweets_emotions.csv', index=False)

In [ ]:
vaccine_tweets_emotions = pd.read_csv(save_root/'vaccine_tweets_emotions.csv')
vaccine_tweets_emotions.created_at = pd.to_datetime(vaccine_tweets_emotions.created_at)

We can make a simple summary of the mined emotions by showing the average, standard deviation, and sum of each emotion column.

In [ ]:
vaccine_tweets_emotions = pd.read_csv(save_root/'vaccine_tweets_emotions.csv')
vaccine_tweets_emotions.created_at = pd.to_datetime(vaccine_tweets_emotions.created_at)
vaccine_tweets_emotions.sort_values('created_at')
emotion_summary = {'avg':[], 'st_dev':[], 'total':[]}
for emotion in labels:
  emotion_summary['avg'].append(vaccine_tweets_emotions[emotion].mean())
  emotion_summary['st_dev'].append(vaccine_tweets_emotions[emotion].std())
  emotion_summary['total'].append(vaccine_tweets_emotions[emotion].sum())
emotion_summary = pd.DataFrame(emotion_summary, index=labels)
emotion_summary

## Preparing vaccination data

The vaccination data from [Our World in Data](https://ourworldindata.org/) contains daily information regarding vaccinations, people vaccinated, and people fully vaccinated. A person is considered vaccinated in the dataset if they have received at least one dose and fully vaccinated if they have received at least two.

To preprocess the vaccination data, we filter the data to only include entries in Japan from December 1, 2021 to December 31, 2021, the same timeframe as the collected tweets. Note that some rows have missing information, which we will have to ingore during our visualizations.

In [ ]:
vaccinations = pd.read_csv('vaccinations.csv')
vaccinations = vaccinations[(vaccinations.iso_code=='JPN') & (vaccinations.date > '2021-12-01') & (vaccinations.date <= '2021-12-31')]
vaccinations.to_csv(save_root/'vaccinations.csv')

In [ ]:
vaccinations = pd.read_csv(save_root/'vaccinations.csv')
vaccinations.date = pd.to_datetime(vaccinations.date)

We will look into three vaccination measures:

* daily vaccinations - the number of doses administered on particular day
* people vaccinated - the number of people who received at least one dose
* people fully vaccinated - the number of people who received at least two doses

In [ ]:
vaccine_stats = ['daily_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']

## Visualizing results

In [ ]:
emotions = ('joy', 'sadness', 'anticipation', 'surprise', 'anger', 'fear', 'disgust', 'trust')
annotators = ('writer', 'reader')
labels = [f'{ann}_{emo}' for ann, emo in product(annotators, emotions)]
writer_labels = [f'writer_{emo}' for emo in emotions]
reader_labels = [f'reader_{emo}' for emo in emotions]

### Distributions of predicted emotion scores for writers and readers

In [ ]:
labels_no_collate = list(chain.from_iterable(zip(writer_labels, reader_labels)))
positions = np.arange(len(emotions))
width = 0.2

fig, ax = plt.subplots()

for sublabels in (writer_labels, reader_labels):
  boxplot = ax.boxplot(
      vaccine_tweets_emotions[sublabels[::-1]],
      showmeans=True,
      sym='',
      vert=False,
      widths=0.4,
      positions=(positions+width) if sublabels==writer_labels else (positions-width),
      patch_artist=True,
      boxprops={'color':'black', 'facecolor':'tab:blue' if sublabels==writer_labels else 'tab:orange'},
      whiskerprops={'color':'black'},
      capprops={'color':'black'},
      medianprops={'color':'black'},
      meanprops={'marker':'.', 'markerfacecolor':'black', 'markeredgecolor':'black'}
  )

writer_patch = mpatches.Patch(color='tab:blue', label='Writer')
reader_patch = mpatches.Patch(color='tab:orange', label='Reader')
ax.legend(handles=[writer_patch, reader_patch])

ax.set_ylabel('Emotion', fontsize=12)
ax.set_xlabel('Score', fontsize=12)
ax.set_yticks(positions, [emo.title() for emo in emotions[::-1]], fontsize=12)
ax.grid(axis='x')

plt.show()

### Distributions of predicted emotions for writers and readers for all keywords and for each keyword

In [ ]:
x = np.arange(len(emotions))
width = 0.4

fig, axs = plt.subplots(3, 2, figsize=(8, 8))
gs = axs[0, 0].get_gridspec()
for ax in axs[0, :]:
  ax.remove()
ax_summary = fig.add_subplot(gs[0, :])

writer_proportions = vaccine_tweets_emotions[writer_labels].sum() / vaccine_tweets_emotions[writer_labels].sum().sum() * 100
reader_proportions = vaccine_tweets_emotions[reader_labels].sum() / vaccine_tweets_emotions[reader_labels].sum().sum() * 100

writer_bars = ax_summary.bar(x - width/2, height=writer_proportions, width=width, label='Writer')
reader_bars = ax_summary.bar(x + width/2, height=reader_proportions, width=width, label='Reader')

ax_summary.set_title('All keywords', fontsize=14)
ax_summary.legend()

keywords_jp = ['ワクチン', 'モデルナ', 'ファイザー', 'オミクロン']
keywords_en = ['Vaccine', 'Moderna', 'Pfizer', 'Omicron']

for keyword_jp, keyword_en, ax in zip(keywords_jp, keywords_en, axs[1:].flat):
  filtered_df = vaccine_tweets_emotions[vaccine_tweets_emotions.text.str.contains(keyword_jp)]
  writer_proportions = filtered_df[writer_labels].sum() / filtered_df[writer_labels].sum().sum() * 100
  reader_proportions = filtered_df[reader_labels].sum() / filtered_df[reader_labels].sum().sum() * 100

  writer_bars = ax.bar(x - width/2, height=writer_proportions, width=width, label='Writer')
  reader_bars = ax.bar(x + width/2, height=reader_proportions, width=width, label='Reader')

  ax.set_title(keyword_en, fontsize=14)

for ax in list(axs[1:].flat)+[ax_summary]:
  ax.set_xticks(x, [emo.title() for emo in emotions], rotation=45, fontsize=14)
  ax.set_yticks(np.arange(0, 35, 5))
  ax.set_axisbelow(True)
  ax.grid(axis='y')

fig.supxlabel('Emotion', fontsize=14)
fig.supylabel('Proportion', fontsize=14)
plt.tight_layout()

plt.show()

### Average score for each emotion and the number of new vaccinations, people vaccinated, and people fully vaccinated for several days in December 2021

In [ ]:
daily_emotion_stats = vaccine_tweets_emotions.groupby(pd.Grouper(key='created_at', freq='D')).mean()
colors = ['orange', 'cornflowerblue', 'pink', 'plum', 'crimson', 'gold', 'seagreen', 'silver']

fig, axs = plt.subplots(3, 2, sharex=True, figsize=(12, 6))
gs = axs[0, 1].get_gridspec()
for ax in axs[:, 0]:
    ax.remove()
ax_emo = fig.add_subplot(gs[:, 0])

for stat, ax in zip(vaccine_stats, axs[:, 1].flat):
  filtered_df = vaccinations[['date', stat]]
  if stat.startswith('people_'):
    filtered_df = filtered_df.diff()
  ids = []
  vals = []
  latest_ids = []
  latest_vals = []
  for i, row in filtered_df.iterrows():
    if not pd.isnull(row[stat]) and not vaccinations.date.loc[i].isoformat().startswith('2021-11-30'):
      latest_vals.append(row[stat])
      latest_ids.append(vaccinations.date.loc[i])
    else:
      if latest_vals:
        vals.append(latest_vals)
        ids.append(latest_ids)
        latest_vals = []
        latest_ids = []
  if latest_vals:
    vals.append(latest_vals)
    ids.append(latest_ids)

  nan_ids = [[ids_subset0[-1], ids_subset1[0]] for ids_subset0, ids_subset1 in zip(ids[:-1], ids[1:])]
  nan_vals = [[vals_subset0[-1], vals_subset1[0]] for vals_subset0, vals_subset1 in zip(vals[:-1], vals[1:])]

  for ids_subset, vals_subset in zip(ids, vals):
    ax.plot(ids_subset, vals_subset, '-|', c='black')
  for ids_subset, vals_subset in zip(nan_ids, nan_vals):
    ax.plot(ids_subset, vals_subset, ':|', c='black')

  ax.set_ylabel(stat.replace('daily_', '').replace('_v', '\nv').replace('_', ' ').title(), fontsize=12)
  ax.tick_params(axis='x', rotation=45, labelsize=12)
  ax.grid()

for i, label in enumerate(labels):
  ax_emo.plot(daily_emotion_stats.index.date, daily_emotion_stats[label], '-' if label.startswith('writer') else ':', color=colors[i%8], label=label.replace('_', ' ').title())

ax_emo.set_ylabel('Average Emotion Score', fontsize=12)
ax_emo.tick_params(axis='x', rotation=45, labelsize=12)
ax_emo.grid()

fig.supxlabel('Date', fontsize=12)
fig.legend(loc='upper left', bbox_to_anchor=(1, 1))
fig.tight_layout()

plt.show()

### Correlation analysis of emotion scores and vaccination statistics

Choose the vaccination measure in the dropdown box below

In [ ]:
daily_emotion_stats = vaccine_tweets_emotions.groupby(pd.Grouper(key='created_at', freq='D')).sum()
daily_emotion_stats.index = daily_emotion_stats.index.tz_localize(None)
daily_vaccine_stats = vaccinations.set_index('date')[vaccine_stats]
data = pd.concat((daily_emotion_stats, daily_vaccine_stats), axis=1)

stat = 'daily_vaccinations' #@param ['daily_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']

fig, axs = plt.subplots(4, 4,  figsize=(10, 10))

for label, ax in zip(labels, axs.flat):
  reformatted_label = label.replace('_', ' ').title()
  reformatted_stat = stat.replace('daily_', '').replace('_', ' ').title()

  filtered_data = data.loc[:, labels+[stat]]
  if stat.startswith('people_'):
    filtered_data[stat] = filtered_data[stat].diff()
  filtered_data = filtered_data.dropna()
  
  slope, intercept = np.polyfit(filtered_data[stat], filtered_data[label], 1)
  endpoint_xs = np.array([filtered_data[stat].min(), filtered_data[stat].max()])

  r, p_value = pearsonr(filtered_data[stat], filtered_data[label])

  ax.scatter(x=filtered_data[stat], y=filtered_data[label], c='tab:green', alpha=0.7)
  ax.plot(endpoint_xs, slope*endpoint_xs+intercept, color='black')

  ax.set_title(f'{reformatted_label}\nr={r:.2f}\np-value={p_value:.3f}', fontsize=18)

  ax.grid()

fig.supxlabel(reformatted_stat, fontsize=24)
fig.supylabel('Total Emotion Score', fontsize=24)
fig.tight_layout()

plt.show()

## Goodness-of-fit tests

### Distributions of emotions between vaccine brands

We conduct a Kolmogorov-Smirnov test to compare the distributions of each writer and reader emotion across two vaccine brands, Moderna and Pfizer.

In [ ]:
moderna_tweets = vaccine_tweets_emotions[vaccine_tweets_emotions.text.str.contains('モデルナ')]
pfizer_tweets = vaccine_tweets_emotions[vaccine_tweets_emotions.text.str.contains('ファイザー')]
ks_results = {'stat':[], 'p':[]}
for label in labels:
  ks_result = ks_2samp(moderna_tweets[label].values, pfizer_tweets[label].values)
  ks_results['stat'].append(ks_result.statistic)
  ks_results['p'].append(ks_result.pvalue)
ks_results = pd.DataFrame(ks_results, index=labels)
ks_results

### Q-Q plots of predicted writer and reader emotions

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(8, 4))

for emotion, ax in zip(emotions, axs.flat):
  writer_subset = vaccine_tweets_emotions[f'writer_{emotion}']
  writer_quantiles = np.array([writer_subset.quantile(i/100) for i in range(100)])
  reader_subset = vaccine_tweets_emotions[f'reader_{emotion}']
  reader_quantiles = np.array([reader_subset.quantile(i/100) for i in range(100)])
  
  ax.plot(writer_quantiles, reader_quantiles, c='black')
  ax.set_xticks([0, 1, 2, 2.5], ['0', '1', '2', ''])
  ax.set_yticks([0, 1, 2, 2.5], ['0', '1', '2', ''])
  ax.grid()
  ax.set_title(emotion.title(), fontsize=14)

fig.supxlabel('Writer Quantiles', fontsize=14)
fig.supylabel('Reader Quantiles', fontsize=14)
fig.tight_layout()

plt.show()